In [2]:
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np 
import re

from matplotlib.axes import Axes
from pandas import DataFrame 
from numpy import ndarray

In [3]:
filenames = [
    f"{typ}-{enc}" 
    for typ in [
        "headon", 
        "overtaking",
        "converging-norm",
        "converging-adverse",
    ]
    for enc in [
        "row",
        "horizontal",
        "vertical"
    ]
]
filenames

['headon-row',
 'headon-horizontal',
 'headon-vertical',
 'overtaking-row',
 'overtaking-horizontal',
 'overtaking-vertical',
 'converging-norm-row',
 'converging-norm-horizontal',
 'converging-norm-vertical',
 'converging-adverse-row',
 'converging-adverse-horizontal',
 'converging-adverse-vertical']

In [4]:
def extract_min_dist(s):
    match = re.search(r'"min_dist"\s*=>\s*([\d\.]+)', s)
    if match:
        return float(match.group(1))
    return None

def get_results(fn: str) -> DataFrame:
    df = pd.read_csv(f"../results/round1/{fn}-results.csv")
    df["severity"] = (2000.0 - df.min_dist) / 2000.0
    # df.loc[df.min_dist > 2000.0, "severity"] = 0.0
    df.loc[df.contactLevel == "none", "severity"] = 0.0
    return df[["contactLevel", "scenario", "drone_response_distance", "min_dist", "severity"]]

def reorder(df: DataFrame, head: list[str]) -> DataFrame:
    return df[[*head, *[c for c in df.columns if c not in head]]]

In [5]:
results = {
    fn: get_results(fn)
    for fn in filenames
}

In [6]:
def compare(
    results: dict[str, DataFrame], 
    title: str
) -> tuple[dict[str, float], dict[str, float]]:
    std = results[f"cs528-{title}-horizontal-params"]
    vrt = results[f"cs528-{title}-vertical-params"]
    std_simcount = std["contactLevel"].count()
    std_viol = std[std.contactLevel != "none"]
    std_n_viol = std_viol["contactLevel"].count()
    std_mean_severity = std_viol.severity.mean()
    std_viol_perc = std_n_viol/std_simcount
    std_risk = std_viol_perc * std_mean_severity
    std_data = {
        "sim_count": std_simcount,
        "n_viol": std_n_viol,
        "mean_severity": std_mean_severity,
        "viol_perc": std_viol_perc,
        "risk": std_risk if not np.isnan(std_risk) else 0.0
    }
    vrt_simcount = vrt["contactLevel"].count()
    vrt_viol = vrt[vrt.contactLevel != "none"]
    vrt_n_viol = vrt_viol["contactLevel"].count()
    vrt_mean_severity = vrt_viol.severity.mean()
    vrt_viol_perc = vrt_n_viol/vrt_simcount
    vrt_risk = vrt_viol_perc * vrt_mean_severity
    vrt_data = {
        "sim_count": vrt_simcount,
        "n_viol": vrt_n_viol,
        "mean_severity": vrt_mean_severity,
        "viol_perc": vrt_viol_perc,
        "risk": vrt_risk if not np.isnan(vrt_risk) else 0.0
    }
    
    return std_data, vrt_data
    
def stats(
    df: DataFrame
) -> dict[str, float]:
    simcount = df["contactLevel"].count()
    viol = df[df.contactLevel != "none"]
    n_viol = viol["contactLevel"].count()
    mean_severity = viol.severity.mean()
    viol_perc = n_viol/simcount
    risk = viol_perc * mean_severity
    
    data = {
        "sim_count": simcount,
        "n_viol": n_viol,
        "mean_severity": mean_severity,
        "viol_perc": viol_perc,
        "risk": risk if not np.isnan(risk) else 0.0
    }
    
    return data

def risk_over_distance(results: dict[str, DataFrame]) -> DataFrame:
    # Returns a dataframe containing risk for each response distance.
    df = None
    
    for title, data in results.items():
        parts = title.split("-")
        scenario = "-".join(parts[:-1])
        maneuver = parts[-1]

        data = data.groupby("drone_response_distance", as_index=False).agg(
            sevsum=("severity", "sum"),
            sim_count=("severity", "size"),
            viol_count=("contactLevel", lambda x: (x != "none").sum())
        ).reset_index()

        data["mean_severity"] = data.sevsum / data.viol_count
        print(data[["sevsum", "viol_count"]])
        data["title"] = title
        data["risk"] = data.viol_count / data.sim_count * data.mean_severity

        data = data.pivot(index="title", columns="drone_response_distance", values="risk")
        data["scenario"] = scenario
        data["maneuver"] = maneuver
        data = reorder(data, ["scenario", "maneuver"])
        data = data.fillna(0.0)
        
        if df is not None:
            df = pd.concat([df, data], ignore_index=True)
        else:
            df = data
    
    return df
        
    
    

In [18]:
[
    (title, stats(df)) for title, df in results.items()   
]

result_summary = None

for title, df in results.items():
    parts = title.split("-")
    maneuver = parts[-1]
    scenario = " ".join(parts[0:-1])
    
    s = DataFrame({key: [val] for key, val in stats(df).items()})
    s["scenario"] = scenario
    s["maneuver"] = maneuver
    s = reorder(s, ["scenario", "maneuver"])
    if result_summary is None:
        result_summary = s 
    else:
        result_summary = pd.concat([result_summary, s])
    
    

# df = risk_over_distance(results)
result_summary.to_csv("tmp2.csv")

In [8]:
df

drone_response_distance,scenario,maneuver,3000.0,3700.0,4400.0,5100.0,5800.0,6500.0,7200.0,7900.0,8600.0,9300.0,10000.0
0,headon,row,0.402460,0.398122,0.396190,0.362254,0.327656,0.273921,0.221700,0.181066,0.135270,0.106536,0.077854
1,headon,horizontal,0.212541,0.153254,0.118712,0.076635,0.051844,0.027965,0.015424,0.006228,0.002605,0.000303,0.000000
2,headon,vertical,0.458174,0.325058,0.278257,0.165223,0.152893,0.151329,0.144716,0.121897,0.071744,0.010798,0.000000
3,overtaking,row,0.407902,0.389894,0.365857,0.317495,0.271211,0.228774,0.177674,0.142854,0.104002,0.080927,0.065650
4,overtaking,horizontal,0.157482,0.111772,0.080548,0.047252,0.030435,0.015703,0.008628,0.002636,0.000340,0.000000,0.000000
5,overtaking,vertical,0.271390,0.150734,0.117021,0.047191,0.010800,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,converging-norm,row,0.181784,0.186593,0.195398,0.218084,0.246244,0.268313,0.276914,0.276914,0.276914,0.276914,0.276914
7,converging-norm,horizontal,0.130730,0.087382,0.059007,0.032818,0.024605,0.020093,0.019859,0.019859,0.019859,0.019859,0.019859
8,converging-norm,vertical,0.217956,0.143942,0.087351,0.073674,0.073364,0.066952,0.054513,0.048100,0.048100,0.048100,0.048100
9,converging-adverse,row,0.192403,0.194786,0.200079,0.225204,0.250924,0.276808,0.278762,0.278762,0.278762,0.278762,0.278762


In [9]:
df.to_csv("tmp.csv")
df

drone_response_distance,scenario,maneuver,3000.0,3700.0,4400.0,5100.0,5800.0,6500.0,7200.0,7900.0,8600.0,9300.0,10000.0
0,headon,row,0.402460,0.398122,0.396190,0.362254,0.327656,0.273921,0.221700,0.181066,0.135270,0.106536,0.077854
1,headon,horizontal,0.212541,0.153254,0.118712,0.076635,0.051844,0.027965,0.015424,0.006228,0.002605,0.000303,0.000000
2,headon,vertical,0.458174,0.325058,0.278257,0.165223,0.152893,0.151329,0.144716,0.121897,0.071744,0.010798,0.000000
3,overtaking,row,0.407902,0.389894,0.365857,0.317495,0.271211,0.228774,0.177674,0.142854,0.104002,0.080927,0.065650
4,overtaking,horizontal,0.157482,0.111772,0.080548,0.047252,0.030435,0.015703,0.008628,0.002636,0.000340,0.000000,0.000000
5,overtaking,vertical,0.271390,0.150734,0.117021,0.047191,0.010800,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,converging-norm,row,0.181784,0.186593,0.195398,0.218084,0.246244,0.268313,0.276914,0.276914,0.276914,0.276914,0.276914
7,converging-norm,horizontal,0.130730,0.087382,0.059007,0.032818,0.024605,0.020093,0.019859,0.019859,0.019859,0.019859,0.019859
8,converging-norm,vertical,0.217956,0.143942,0.087351,0.073674,0.073364,0.066952,0.054513,0.048100,0.048100,0.048100,0.048100
9,converging-adverse,row,0.192403,0.194786,0.200079,0.225204,0.250924,0.276808,0.278762,0.278762,0.278762,0.278762,0.278762


In [10]:
compare(results, "converging-adverse")

KeyError: 'cs528-converging-adverse-horizontal-params'